In [2]:
import pandas as pd
import numpy as np

In [3]:
import calendar

In [4]:
from datetime import datetime
import time

In [5]:
import os

In [6]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
sns.set()

In [7]:
selection = 25
year = '2019'
airline = 'DL'
download_path = r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data'
file_path = os.path.join(download_path, f'{selection}airports{airline}{year}.csv')

In [8]:
download_path = r'/home/desbrium/Metis/PredictingFlightDelays/Data/BTS Departure Data'

In [9]:
file_path = os.path.join(download_path, f'{selection}airports{airline}{year}.csv')

In [10]:
df = pd.read_csv(file_path)

In [163]:
Region_Division = {'Northeast':['New Englad','Middle Atlantic'], 
                   'Midwest':['East North Central','West North Central'], 
                   'South': ['South Atlantic','East South Central','West South Central'], 
                   'West': ['Mountain','Pacific'],
                   'U.S. Territory': ['Island']
                  }

In [164]:
RD_list = []

In [165]:
for region, divisions in Region_Division.items():
    for division in divisions:
        RD_dict = {}
        RD_dict['Region'] = region
        RD_dict['Division'] = division
        RD_list.append(RD_dict)

In [166]:
RD_list

[{'Region': 'Northeast', 'Division': 'New Englad'},
 {'Region': 'Northeast', 'Division': 'Middle Atlantic'},
 {'Region': 'Midwest', 'Division': 'East North Central'},
 {'Region': 'Midwest', 'Division': 'West North Central'},
 {'Region': 'South', 'Division': 'South Atlantic'},
 {'Region': 'South', 'Division': 'East South Central'},
 {'Region': 'South', 'Division': 'West South Central'},
 {'Region': 'West', 'Division': 'Mountain'},
 {'Region': 'West', 'Division': 'Pacific'},
 {'Region': 'U.S. Territory', 'Division': 'Island'}]

In [167]:
regions_df = pd.DataFrame(RD_list)

In [168]:
Division_State = {'New Englad':['Maine','New Hampshire','Vermont','Massachusetts','Rhode Island','Connecticut'], 
                  'Middle Atlantic':['New York','Pennsylvania','New Jersey'], 
                  'East North Central':['Wisconsin','Michigan','Illinois','Indiana','Ohio'],
                  'West North Central':['North Dakota','South Dakota','Nebraska','Kansas','Minnesota','Iowa','Missouri'], 
                  'South Atlantic':['Delaware','Maryland','District of Columbia','Virginia','West Virginia','North Carolina','South Carolina','Georgia','Florida'],
                  'East South Central':['Kentucky','Tennessee','Mississippi','Alabama'],
                  'West South Central':['Oklahoma','Texas','Arkansas','Louisiana'], 
                  'Mountain':['Idaho','Montana','Wyoming','Nevada','Utah','Colorado','Arizona','New Mexico'],
                  'Pacific':['Alaska','Washington','Oregon','California','Hawaii'],
                  'Island': ['Puerto Rico', 'U.S. Virgin Islands']
                 }

In [169]:
DS_list = []

In [170]:
for division, states in Division_State.items():
    for state in states:
        DS_dict= {}
        DS_dict['Division'] = division
        DS_dict['State'] = state
        DS_list.append(DS_dict)

In [171]:
state_df = pd.DataFrame(DS_list)

In [172]:
regions_df

,Region,Division
0,Northeast,New Englad
1,Northeast,Middle Atlantic
2,Midwest,East North Central
3,Midwest,West North Central
4,South,South Atlantic
5,South,East South Central
6,South,West South Central
7,West,Mountain
8,West,Pacific
9,U.S. Territory,Island


In [173]:
regions_df = state_df.merge(regions_df, how='left', on= 'Division')

In [174]:
regions_df[['Origin State', 'Origin Divison', 'Origin Region']] = regions_df[['State', 'Division', 'Region']]

In [175]:
regions_df[['Dest State', 'Dest Divison', 'Dest Region']] = regions_df[['State', 'Division', 'Region']]

In [176]:
origin_regions_df = regions_df[['Origin State', 'Origin Divison', 'Origin Region']]

In [177]:
dest_regions_df = regions_df[['Dest State', 'Dest Divison', 'Dest Region']]

In [178]:
df = df.merge(origin_regions_df, how='left', on='Origin State')

In [179]:
df = df.merge(dest_regions_df, how='left', on='Dest State')

In [229]:
def time_bin(time):
    
    if time >= '04:00' and time < '08:00':

        time = 'Early Morning'

        return time

    elif time >= '08:00' and time < '12:00':

        time = 'Morning'

        return time

    elif time >= '12:00' and time < '16:00':

        time = 'Afternoon'

        return time

    elif time >= '16:00' and time < '20:00':

        time = 'Evening'

        return time

    elif time >= '20:00' and time < '24:00':

        time = 'Night'

        return time

    else:

        time = 'Late Night'

        return time


In [121]:
df['Actual Departure Time Name'] = df['Actual Departure Time'].apply(lambda time: time_bin(time))

In [148]:
df['Schedule Departure Time Name'] = df['Scheduled Departure Time'].apply(lambda time: time_bin(time))

In [230]:
df['Schedule Arrival Time Name'] = df['Scheduled Arrival Time'].apply(lambda time: time_bin(time))

In [149]:
bins = [90, 583, 1072, 1561, 2050, 2539, 3027, 3516, 4005, 4494, 4983]
names = ['90-583', '583-1072', '1072-1561', '1561-2050', '2050-2539', '2539-3027', '3027-3516', '3516-4005','4005-4494', '4494-4983']

In [150]:
df['Distance Range Name'] = pd.cut(df['Distance'],bins, labels=names)

In [196]:
bins = [0, 3000, np.inf]
names = ['Short Haul', 'Long Haul']

df['Distance Name'] = pd.cut(df['Distance'], bins, labels=names)

In [9]:
df['Flight Date'] = pd.to_datetime(df['Flight Date'], format = '%m/%d/%Y')

In [165]:
df['Month'] = df['Flight Date'].dt.month

In [221]:
df['Month'] = df['Month'].astype(int).apply(lambda month: calendar.month_name[month])

In [151]:
bins = [90, 583, 1072, 1561, 2050, 2539, 3027, 3516, 4005, 4494, 4983]
names = ['90-583', '583-1072', '1072-1561', '1561-2050', '2050-2539', '2539-3027', '3027-3516', '3516-4005','4005-4494', '4494-4983']

In [152]:
df['Distance Range Name'] = pd.cut(df['Distance'],bins, labels=names)

In [153]:
def convert_int_to_time(x):
    
    x = list(x)
    x.insert(-2,':')
    x = ''.join(x)
    
    return x

In [154]:
df['Scheduled Arrival Time'] = df['Scheduled Arrival Time'].astype(str).apply(lambda x: convert_int_to_time(x))

In [155]:
test_df = df[['Flight Date', 'Tail Number', 'Scheduled Departure Time']].groupby(['Flight Date', 'Tail Number'], as_index = False).agg({'Scheduled Departure Time': 'count'}).sort_values('Scheduled Departure Time', ascending = False)

In [156]:
test_df.rename(columns = {'Scheduled Departure Time' : 'Total Number of Scheduled Flights'}, inplace = True)

In [157]:
df = df.merge(test_df, how = 'left', on = ['Flight Date', 'Tail Number'])

In [34]:
testdf1 = df[['Flight Date', 'Tail Number','Minutes Delayed By Carrier', 'Minutes Delayed By NAS',
              'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival']].groupby(['Tail Number', 'Flight Date']).mean().round(2).apply(lambda grp: grp.shift(1))
testdf1 = testdf1.rename(columns = {'Minutes Delayed By Carrier': 'Avg Min Delayed Due To Delta Yesterday','Minutes Delayed By NAS': 'Avg Min Delayed Due To NAS Yesterday', 
                                    'Minutes Delayed By Security': 'Avg Min Delayed Due To Security Yesterday',
                                    'Minutes Delayed By Late Arrival': 'Avg Min Delayed Due To Late Arrival Yesterday'}).reset_index()
testdf1.fillna(value = 0, inplace = True)

In [28]:
test_df = df[['Flight Date', 'Origin Airport', 'Scheduled Departure Time']].groupby(['Flight Date', 'Origin Airport'], as_index = False).agg({'Scheduled Departure Time': 'count'}).sort_values('Scheduled Departure Time', ascending = False)
test_df.rename(columns = {'Scheduled Departure Time' : 'Total Number of Scheduled Flights Airport'}, inplace = True)
df = df.merge(test_df, how = 'left', on = ['Flight Date', 'Origin Airport'])

In [23]:
testdf2 = df[['Flight Date', 'Origin Airport','Minutes Delayed By Carrier', 'Minutes Delayed By NAS',
              'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival', 'Minutes Delayed By Weather']].groupby(['Origin Airport','Flight Date']).mean().round(2).apply(lambda grp: grp.shift(1))
testdf2 = testdf2.rename(columns = {'Minutes Delayed By Weather': 'Avg Min Delayed Due To Weather Yesterday Airport', 
                                    'Minutes Delayed By Carrier': 'Avg Min Delayed Due To Delta Yesterday Airport',
                                    'Minutes Delayed By NAS': 'Avg Min Delayed Due To NAS Yesterday Airport', 
                                    'Minutes Delayed By Security': 'Avg Min Delayed Due To Security Yesterday Airport',
                                    'Minutes Delayed By Late Arrival': 'Avg Min Delayed Due To Late Arrival Yesterday Airport'}).reset_index()
testdf2.fillna(value = 0, inplace = True)

In [24]:
df = df.merge(testdf2, how= 'left', on = ['Flight Date', 'Origin Airport'])

In [35]:
df = df.merge(testdf1, how= 'left', on = ['Flight Date', 'Tail Number'])

In [ ]:
#testdf2 = df[['Flight Date', 'Origin Region', 'Dest Region','Minutes Delayed By Carrier', 'Minutes Delayed By NAS',
              #'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival', 'Minutes Delayed By Weather']].groupby(['Origin Region', 'Dest Region', 'Flight Date']).mean().round(2).apply(lambda grp: grp.shift(1))
#testdf2 = testdf2.rename(columns = {'Minutes Delayed By Weather': 'Avg Min Delayed Due To Weather Yesterday Region', 
                                    #'Minutes Delayed By Carrier': 'Avg Min Delayed Due To Delta Yesterday Region',
                                    #'Minutes Delayed By NAS': 'Avg Min Delayed Due To NAS Yesterday Region', 
                                    #'Minutes Delayed By Security': 'Avg Min Delayed Due To Security Yesterday Region',
                                    #'Minutes Delayed By Late Arrival': 'Avg Min Delayed Due To Late Arrival Yesterday Region'}).reset_index()
#testdf2.fillna(value = 0, inplace = True)

In [14]:
#df = df.merge(testdf2, how= 'left', on = ['Flight Date', 'Origin Region', 'Dest Region'])

In [32]:
df = df[['Unique_Id', 'Carrier Code', 'Flight Date', 'Flight Number', 'Month',
       'Tail Number', 'Origin Airport', 'Origin City', 'Origin State',
       'Origin Divison', 'Origin Region', 'Dest Airport', 'Dest City',
       'Dest State', 'Dest Divison', 'Dest Region', 'Scheduled Departure Time',
       'Schedule Departure Time Name', 'Actual Departure Time',
       'Actual Departure Time Name', 'Minutes Delayed Departing',
       'Delayed Departure', 'Scheduled Elapsed Time', 'Actual Elapsed Time',
       'Wheels-off Time', 'Taxi-out Time', 'Minutes Delayed By Carrier',
       'Minutes Delayed By Weather', 'Minutes Delayed By NAS',
       'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival',
       'Total Number of Scheduled Flights', 'Cancelled', 'Diverted', 'Flights',
       'Distance', 'Distance Name', 'Distance Range Name',
       'Scheduled Arrival Time', 'Schedule Arrival Time Name',
       'Actual Arrival Time', 'Minutes Delayed Arriving', 'AirTime',
       'Avg Min Delayed Due To Delta Yesterday Region',
       'Avg Min Delayed Due To NAS Yesterday Region',
       'Avg Min Delayed Due To Security Yesterday Region',
       'Avg Min Delayed Due To Late Arrival Yesterday Region',
       'Avg Min Delayed Due To Weather Yesterday Region',
       'Avg Min Delayed Due To Delta Yesterday Airport',
       'Avg Min Delayed Due To NAS Yesterday Airport',
       'Avg Min Delayed Due To Security Yesterday Airport',
       'Avg Min Delayed Due To Late Arrival Yesterday Airport',
       'Avg Min Delayed Due To Weather Yesterday Airport',
        'Total Number of Scheduled Flights Airport']]

In [33]:
df.columns

Index(['Unique_Id', 'Carrier Code', 'Flight Date', 'Flight Number', 'Month',
       'Tail Number', 'Origin Airport', 'Origin City', 'Origin State',
       'Origin Divison', 'Origin Region', 'Dest Airport', 'Dest City',
       'Dest State', 'Dest Divison', 'Dest Region', 'Scheduled Departure Time',
       'Schedule Departure Time Name', 'Actual Departure Time',
       'Actual Departure Time Name', 'Minutes Delayed Departing',
       'Delayed Departure', 'Scheduled Elapsed Time', 'Actual Elapsed Time',
       'Wheels-off Time', 'Taxi-out Time', 'Minutes Delayed By Carrier',
       'Minutes Delayed By Weather', 'Minutes Delayed By NAS',
       'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival',
       'Total Number of Scheduled Flights', 'Cancelled', 'Diverted', 'Flights',
       'Distance', 'Distance Name', 'Distance Range Name',
       'Scheduled Arrival Time', 'Schedule Arrival Time Name',
       'Actual Arrival Time', 'Minutes Delayed Arriving', 'AirTime',
       'Avg Mi

In [16]:
df = df[['Unique_Id', 'Carrier Code', 'Flight Date', 'Flight Number', 'Month',
       'Tail Number', 'Origin Airport', 'Origin City', 'Origin State',
       'Origin Divison', 'Origin Region', 'Dest Airport', 'Dest City',
       'Dest State', 'Dest Divison', 'Dest Region', 'Scheduled Departure Time',
       'Schedule Departure Time Name', 'Actual Departure Time',
       'Actual Departure Time Name', 'Minutes Delayed Departing',
       'Delayed Departure', 'Scheduled Elapsed Time', 'Actual Elapsed Time',
       'Wheels-off Time', 'Taxi-out Time', 'Minutes Delayed By Carrier',
       'Minutes Delayed By Weather', 'Minutes Delayed By NAS',
       'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival',
       'Total Number of Scheduled Flights', 'Cancelled', 'Diverted', 'Flights',
       'Distance', 'Distance Name', 'Distance Range Name',
       'Scheduled Arrival Time', 'Schedule Arrival Time Name',
       'Actual Arrival Time', 'Minutes Delayed Arriving', 'AirTime',
       'Total Number of Scheduled Flights Region',
       'Avg Min Delayed Due To Delta Yesterday Region',
       'Avg Min Delayed Due To NAS Yesterday Region',
       'Avg Min Delayed Due To Security Yesterday Region',
       'Avg Min Delayed Due To Late Arrival Yesterday Region',
       'Avg Min Delayed Due To Weather Yesterday Region']]

In [246]:
def edit_time(x):
    if x.find(':') == 1:
        x = '0' + x
    if x.find(':') == 0:
        x = '00' + x
    return x

In [248]:
df['Scheduled Arrival Time'] = df['Scheduled Arrival Time'].apply(lambda x: edit_time(x))

In [249]:
df['Scheduled Departure Time'] = df['Scheduled Departure Time'].apply(lambda x: datetime.strptime(x, '%H:%M').time())
df['Scheduled Arrival Time'] = df['Scheduled Arrival Time'].apply(lambda x: datetime.strptime(x, '%H:%M').time())

In [36]:
df.to_csv(file_path, index = False)

In [26]:
df.columns

Index(['Unique_Id', 'Carrier Code', 'Flight Date', 'Flight Number', 'Month',
       'Tail Number', 'Origin Airport', 'Origin City', 'Origin State',
       'Origin Divison', 'Origin Region', 'Dest Airport', 'Dest City',
       'Dest State', 'Dest Divison', 'Dest Region', 'Scheduled Departure Time',
       'Schedule Departure Time Name', 'Actual Departure Time',
       'Actual Departure Time Name', 'Minutes Delayed Departing',
       'Delayed Departure', 'Scheduled Elapsed Time', 'Actual Elapsed Time',
       'Wheels-off Time', 'Taxi-out Time', 'Minutes Delayed By Carrier',
       'Minutes Delayed By Weather', 'Minutes Delayed By NAS',
       'Minutes Delayed By Security', 'Minutes Delayed By Late Arrival',
       'Total Number of Scheduled Flights', 'Cancelled', 'Diverted', 'Flights',
       'Distance', 'Distance Name', 'Distance Range Name',
       'Scheduled Arrival Time', 'Schedule Arrival Time Name',
       'Actual Arrival Time', 'Minutes Delayed Arriving', 'AirTime',
       'Total 